<a href="https://colab.research.google.com/github/bhumz123/project/blob/main/nlp_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q peft
!pip install -q evaluate

In [ ]:
from datasets import load_dataset
dataset=load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

### Remove Punctuation

In [ ]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
import string
def remove_punctuation(text):
    ch_text="".join([i for i in text if i not in string.punctuation])
    return ch_text

In [ ]:
for split in dataset.keys():
    dataset[split] = dataset[split].map(lambda example: {'text': remove_punctuation(example['text']), 'label': example['label']})


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
print(dataset['train'][3])

{'text': 'This film was probably inspired by Godards Masculin féminin and I urge you to see that film insteadbr br The film has two strong elements and those are 1 the realistic acting 2 the impressive undeservedly good photo Apart from that what strikes me most is the endless stream of silliness Lena Nyman has to be most annoying actress in the world She acts so stupid and with all the nudity in this filmits unattractive Comparing to Godards film intellectuality has been replaced with stupidity Without going too far on this subject I would say that follows from the difference in ideals between the French and the Swedish societybr br A movie of its time and place 210', 'label': 0}


### Removing Stop Words

In [ ]:
import nltk
stopwords=nltk.corpus.stopwords.words('english')

In [ ]:
def remove_stopwords(text):
    tokens=nltk.word_tokenize(text)
    output=" ".join(i for i in tokens if i not in stopwords)
    return output

In [ ]:
for split in dataset.keys():
    dataset[split] = dataset[split].map(lambda example: {'text': remove_stopwords(example['text']), 'label': example['label']})

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
print(dataset['train'][3])

{'text': 'This film probably inspired Godards Masculin féminin I urge see film insteadbr br The film two strong elements 1 realistic acting 2 impressive undeservedly good photo Apart strikes endless stream silliness Lena Nyman annoying actress world She acts stupid nudity filmits unattractive Comparing Godards film intellectuality replaced stupidity Without going far subject I would say follows difference ideals French Swedish societybr br A movie time place 210', 'label': 0}


### Tokenization

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
from peft import LoraConfig, TaskType


In [ ]:
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=1, lora_alpha=1, lora_dropout=0.1
)

In [ ]:
from transformers import BertForSequenceClassification

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-cased',
    num_labels=2
)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from peft import get_peft_model
model = get_peft_model(model, lora_config)

In [ ]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(28996, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): L

In [ ]:
# Count trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

# Count all parameters
all_params = sum(p.numel() for p in model.parameters())

# Calculate trainable percentage
trainable_percentage = trainable_params / all_params

# Print the results
print(f"Trainable params: {trainable_params} || All params: {all_params} || Trainable %: {trainable_percentage}")

Trainable params: 38402 || All params: 108350212 || Trainable %: 0.0003544247795288116


In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

2024-05-07 07:15:54.993425: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-07 07:15:54.993527: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-07 07:15:55.125527: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer",
                                  evaluation_strategy="epoch",
                                   report_to="none",
                                 num_train_epochs=10)

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.686869,0.542000
2,No log,0.682450,0.546000
3,No log,0.679834,0.547000
4,0.694000,0.680991,0.525000
5,0.694000,0.672125,0.647000
6,0.694000,0.665991,0.661000
7,0.694000,0.661955,0.641000
8,0.679800,0.658930,0.641000
9,0.679800,0.656617,0.640000
10,0.679800,0.656307,0.641000


TrainOutput(global_step=1250, training_loss=0.6840779174804688, metrics={'train_runtime': 614.7712, 'train_samples_per_second': 16.266, 'train_steps_per_second': 2.033, 'total_flos': 2632290263040000.0, 'train_loss': 0.6840779174804688, 'epoch': 10.0})

In [ ]:
text="Had my mind blown by the film 'Animal'! Ranbir Kapoor is on another level, redefining the game with his acting prowess. He brings a perfect blend of swag, emotions, good looks and upholds the Kapoor legacy. Sandeep Reddy Vanga, the Kabir Singh magician, does it again! "

In [ ]:
ttext=remove_punctuation(text)

In [ ]:
mtext=remove_stopwords(ttext)

In [ ]:
trained_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
tokenized_input=trained_tokenizer(mtext, padding="max_length", truncation=True,return_tensors='pt')

In [ ]:
trained_model = BertForSequenceClassification.from_pretrained('/kaggle/working/test_trainer/checkpoint-1000')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch

In [ ]:
tokenized_input

{'input_ids': tensor([[  101,  6467,  1713, 10816,  1273, 10854, 16890,  1179, 18710, 18321,
          1330,  1634,  1894, 11470, 16534,  1342,  3176,  5250, 14291,  1116,
          1124,  7100,  3264, 13390,   188,  3624,  1403,  6288,  1363,  2736,
          1146,  8678,  1116, 18321, 10261, 16377, 17059, 20772,  3605,  2571,
         14812, 18710,  5329, 22334,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [ ]:
# Ensure model is in evaluation mode
trained_model.eval()

# Forward pass, get logits
with torch.no_grad():
    outputs = trained_model(**tokenized_input)



In [ ]:
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-0.4882,  0.0844]]), hidden_states=None, attentions=None)

In [ ]:
import torch.nn.functional as F

In [ ]:
logits = torch.tensor([[-0.4882, 0.0844]])

# Apply softmax to logits
probabilities = F.softmax(logits, dim=1)

print("Probabilities:", probabilities)

Probabilities: tensor([[0.3606, 0.6394]])


In [ ]:
predicted_class = torch.argmax(probabilities, dim=1)
predicted_class_index = predicted_class.item()

# Assuming class labels (0 and 1)
class_labels = ['Negative', 'Positive']
predicted_class_label = class_labels[predicted_class_index]

print("Predicted class:", predicted_class_label)

Predicted class: Positive
